**20MAI1013**     **fashion_mnist** **Dataset** **using** **AlexNet** **architecture** 

In [1]:
#Keras library for CIFAR dataset
from keras.datasets import fashion_mnist

from keras.callbacks import History
history = History()


In [2]:
(x_train, y_train),(x_test, y_test)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [3]:
#Train-validation-test split
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

In [5]:
x_train = x_train.reshape(-1,28,28,1)
x_val = x_val.reshape(-1,28,28,1)

In [6]:
#Dimension of the CIFAR10 dataset
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
#print((x_test.shape,y_test.shape))

((42000, 28, 28, 1), (42000,))
((18000, 28, 28, 1), (18000,))


In [7]:
#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
#y_test=to_categorical(y_test)



In [8]:
#Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
#print((x_test.shape,y_test.shape))

((42000, 28, 28, 1), (42000, 10))
((18000, 28, 28, 1), (18000, 10))


In [9]:
#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
#test_generator.fit(x_test)

In [10]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

In [11]:
np.random.seed(1000)

#Instantiation
AlexNet = Sequential()

In [12]:
#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(28,28,1), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(28,28,1)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(10))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))


In [13]:
#Model Summary
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 7, 96)          11712     
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 96)          384       
_________________________________________________________________
activation (Activation)      (None, 7, 7, 96)          0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 4, 256)         0

In [14]:
#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(   monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5) 

In [15]:
#Defining the parameters
batch_size= 100
epochs=60
learn_rate=.01

In [16]:
# Compiling the model
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [17]:
#Training the model
history = AlexNet.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = 250, callbacks = [lrr], verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
420/420 [==============================] - 50s 37ms/step - loss: 0.9203 - accuracy: 0.7293 - val_loss: 1.3135 - val_accuracy: 0.7201
Epoch 2/60
420/420 [==============================] - 12s 29ms/step - loss: 0.5634 - accuracy: 0.8332
Epoch 3/60
420/420 [==============================] - 12s 29ms/step - loss: 0.4803 - accuracy: 0.8484
Epoch 4/60
420/420 [==============================] - 12s 28ms/step - loss: 0.4224 - accuracy: 0.8607
Epoch 5/60
420/420 [==============================] - 12s 29ms/step - loss: 0.4000 - accuracy: 0.8675
Epoch 6/60
420/420 [==============================] - 12s 29ms/step - loss: 0.3789 - accuracy: 0.8725
Epoch 7/60
420/420 [==============================] - 12s 28ms/step - loss: 0.3638 - accuracy: 0.8765
Epoch 8/60
420/420 [==============================] - 12s 29ms/step - loss: 0.3480 - accuracy: 0.8791
Epoch 9/60
420/420 [==============================] - 12s 28ms/step - loss: 0.3430 - accuracy: 0.8808
Epoch 10/60
420/420 [===================

In [18]:
loss,acc = AlexNet.evaluate(x_val,y_val)
print("Accuracy: %.3f" %acc)
print("Loss: %.3f" %loss)

563/563 [==============================] - 3s 5ms/step - loss: 0.4157 - accuracy: 0.8673
Accuracy: 0.867
Loss: 0.416


In this model Implementation of fashion_mnist using AlexNet architecture is done.
Libraries used for implementation are:
Keras, tensorflow, ImageDataGenerator.

Some layers used: Dense,Flatten,Convolution2D,Dropout,MaxPooling.

Parameters: Strides, KernelSize, Input size, activation functions, hidden layers, number of neurons, dimensions.



Observation:

Train Accuracy: 94.83%

Loss : 0.1361


Test Accuracy: 86.73%

Loss: 0.416